In [1]:
from jupyter_dash import JupyterDash

In [2]:
import dash
import dash_html_components as html
from dash_jbrowse import LinearGenomeView

In [3]:
def launch_jbrowse(assembly, tracks, defaultSession, location, aggregateTextSearchAdapters):
    app = JupyterDash(__name__)

    app.layout = html.Div(
        [
            LinearGenomeView(
                id="input",
                assembly=assembly,
                tracks=tracks,
                defaultSession=defaultSession,
                location=location,
                aggregateTextSearchAdapters=aggregateTextSearchAdapters
            ),
        ],
    )
    
    app.run_server(mode="inline",height=300, port=3000)

In [4]:
my_assembly = {
    "name": "GRCh38",
    "sequence": {
        "type": "ReferenceSequenceTrack",
        "trackId": "GRCh38-ReferenceSequenceTrack",
        "adapter": {
            "type": "BgzipFastaAdapter",
            "fastaLocation": {
                "uri": "https://s3.amazonaws.com/jbrowse.org/genomes/GRCh38/fasta/GRCh38.fa.gz",
            },
            "faiLocation": {
                "uri": "https://s3.amazonaws.com/jbrowse.org/genomes/GRCh38/fasta/GRCh38.fa.gz.fai",
            },
            "gziLocation": {
                "uri": "https://s3.amazonaws.com/jbrowse.org/genomes/GRCh38/fasta/GRCh38.fa.gz.gzi",
            },
        },
    },
    "aliases": ["hg38"],
    "refNameAliases": {
        "adapter": {
            "type": "RefNameAliasAdapter",
            "location": {
                "uri": "https://s3.amazonaws.com/jbrowse.org/genomes/GRCh38/hg38_aliases.txt",
            },
        },
    },
}

my_tracks = [
    {
        "type": "FeatureTrack",
        "trackId": "ncbi_refseq_109_hg38",
        "name": "NCBI RefSeq (GFF3Tabix)",
        "assemblyNames": ["GRCh38"],
        "category": ["Annotation"],
        "adapter": {
            "type": "Gff3TabixAdapter",
            "gffGzLocation": {
                "uri": "https://s3.amazonaws.com/jbrowse.org/genomes/GRCh38/ncbi_refseq/GCA_000001405.15_GRCh38_full_analysis_set.refseq_annotation.sorted.gff.gz",
            },
            "index": {
                "location": {
                    "uri": "https://s3.amazonaws.com/jbrowse.org/genomes/GRCh38/ncbi_refseq/GCA_000001405.15_GRCh38_full_analysis_set.refseq_annotation.sorted.gff.gz.tbi",
                },
            },
        },
    },
]

my_default_session = {
    "name": "My session",
    "view": {
        "id": "linearGenomeView",
        "type": "LinearGenomeView",
        "tracks": [
            {
                "type": "FeatureTrack",
                "configuration": "ncbi_refseq_109_hg38",
                "displays": [
                    {
                        "type": "LinearBasicDisplay",
                        "configuration": "ncbi_refseq_109_hg38-LinearBasicDisplay",
                    },
                ],
            },
        ],
    },
}
my_aggregate_text_search_adapters=[
    {
        "type": "TrixTextSearchAdapter",
        "textSearchAdapterId": "hg38-index",
        "ixFilePath": {
            "uri": "https://s3.amazonaws.com/jbrowse.org/genomes/GRCh38/trix/hg38.ix"
        },
        "ixxFilePath": {
            "uri": "https://s3.amazonaws.com/jbrowse.org/genomes/GRCh38/trix/hg38.ixx"
        },
        "metaFilePath": {
            "uri": "https://s3.amazonaws.com/jbrowse.org/genomes/GRCh38/trix/meta.json"
        },
        "assemblyNames": ['GRCh38'],
    }
]
my_location = "7:140,713,328-140,924,929"


In [5]:
launch_jbrowse(my_assembly, my_tracks, my_default_session, my_location, my_aggregate_text_search_adapters)

In [6]:
import dash
import dash_html_components as html
from dash_jbrowse import CircularGenomeView
from jupyter_dash import JupyterDash

In [7]:
def launch_jbrowse_cvg(assembly, tracks, defaultSession, aggregateTextSearchAdapters):
    app = JupyterDash(__name__)

    app.layout = html.Div(
        [
            CircularGenomeView(
                id="input",
                assembly=assembly,
                tracks=tracks,
                defaultSession=defaultSession,
                aggregateTextSearchAdapters=aggregateTextSearchAdapters
            ),
        ],
    )
    
    app.run_server(mode="inline",height=600, port=3002)

In [8]:
my_assembly = {
    "name": "hg19",
    "aliases": ["GRCh37"],
    "sequence": {
        "type": "ReferenceSequenceTrack",
        "trackId": "Pd8Wh30ei9R",
        "adapter": {
            "type": "BgzipFastaAdapter",
            "fastaLocation": {
                "uri": "https://jbrowse.org/genomes/hg19/fasta/hg19.fa.gz",
                "locationType": 'UriLocation',
            },
            "faiLocation": {
                "uri": "https://jbrowse.org/genomes/hg19/fasta/hg19.fa.gz.fai",
                "locationType": 'UriLocation',
            },
            "gziLocation": {
                "uri": "https://jbrowse.org/genomes/hg19/fasta/hg19.fa.gz.gzi",
                "locationType": 'UriLocation',
            },
        },
    },
    "refNameAliases": {
        "adapter": {
            "type": "RefNameAliasAdapter",
            "location": {
                "uri": "https://s3.amazonaws.com/jbrowse.org/genomes/hg19/hg19_aliases.txt",
                "locationType": 'UriLocation',
            },
        },
    },
}

my_tracks = [
    {
        "type": "VariantTrack",
        "trackId": "pacbio_sv_vcf",
        "name": "HG002 Pacbio SV (VCF)",
        "assemblyNames": ["hg19"],
        "category": ["GIAB"],
        "adapter": {
            "type": "VcfTabixAdapter",
            "vcfGzLocation": {
                "uri": 'https://s3.amazonaws.com/jbrowse.org/genomes/hg19/pacbio/hs37d5.HG002-SequelII-CCS.bnd-only.sv.vcf.gz',
                "locationType": 'UriLocation',
            },
            "index": {
                "location": {
                    "uri": 'https://s3.amazonaws.com/jbrowse.org/genomes/hg19/pacbio/hs37d5.HG002-SequelII-CCS.bnd-only.sv.vcf.gz.tbi',
                    "locationType": 'UriLocation',
                },
            },
        },
    },
]

my_default_session = {
    "name": "My session",
    "view": {
        "id": "circularView",
        "type": "CircularView",
        "bpPerPx": 5000000, 
    },
}


my_theme = {
    "theme": {
        "palette": {
            "primary": {
                "main": "#311b92",
            },
            "secondary": {
                "main": "#0097a7",
            },
            "tertiary": {
                "main": "#f57c00",
            },
            "quaternary": {
                "main": "#d50000",
            },
            "bases": {
                "A": {"main": "#98FB98"},
                "C": {"main": "#87CEEB"},
                "G": {"main": "#DAA520"},
                "T": {"main": "#DC143C"},
            },
        },
    },
}


In [9]:
launch_jbrowse_cvg(my_assembly, my_tracks, my_default_session, [])